<a href="https://colab.research.google.com/github/Shambhaviadhikari/PythonClass/blob/main/GW_Events_Scraper_Challenge!_(Fall_2024)_%5BG37903602%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📅 GWU Events Scraper Challenge 📅


## Instructions

For this exercise, we will scrape events from the [GW events calendar](https://calendar.gwu.edu/calendar) website.


It may be helpful to take a few minutes to inspect the website to learn more, and make a plan, before developing a solution.

Feel free to use the "Scratch Work" section at the bottom (which will not be evaluated). Then when you are satisfied, return to implement your final code in the "Solution" section.

## Evaluation

Rubric (tentative):

Part | Question | Weight
--- | --- | ---
Part I | Question 1 (Fetch Events) | 10%
Part I | Question 2 (Display Events) | 35%
Part I | Question 3 (Collect Events) | 10%
Part I | Questions 4 and 5 (Save Events) | 10%
Part II | Question 6 (Custom Function) | 20%
Part II | Question 7 (Tests) | 15%



## Part I

### Fetch Events



1) Request the **HTML** page contents of https://calendar.gwu.edu/calendar and create a `BeautifulSoup` object with the resulting page contents.

In [ ]:


EVENTS_PAGE_URL = "https://calendar.gwu.edu/calendar"





In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the events page
EVENTS_PAGE_URL = "https://calendar.gwu.edu/calendar"

# Request the HTML page content
response = requests.get(EVENTS_PAGE_URL)

# Create a BeautifulSoup object with the page contents
soup = BeautifulSoup(response.content, 'html.parser')

# Optional: Print the title of the page to verify
print(soup.title.text)


  Events  on October 17, 2024 - April 16, 2025 - GW Calendar


### Display and Collect Events



2-A) Parse the page and loop through the events and **display** some information about each, including:
  + **Title** (e.g. "Annie Artist Gallery Opening")
  + **Date info** including date, day of week, start, and end times (e.g. "Mon, Oct 2, 2023 11am to 3pm" or "Mon, Oct 2, 2023", depending on whether or not the start and end times are present - some events don't have start and end)
  + **Location** (e.g. "Smith Hall of Art, Gallery 102")
  + **Image** displayed
  + **Primary event type** / first tag (e.g. "Arts and Culture")

2-B) Revisit 2-A, and see if you can further **decompose the date info** into multiple separate fields:
  + **Date** (e.g. "2023-09-08")
  + **Day of week** (e.g. "Friday")
  + **Start time**, as applicable (e.g. "12:30pm" or None)
  + **End time**, as applicable (e.g. "01:30pm" or None)

3) Ensure your loop from Question 2 also **collects** the events as a list of dictionaries called `events`.


In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Image
import re
import datetime


In [ ]:

# List to hold all events
events = []

# Loop through event containers
for event in soup.find_all(class_="em-card"):
    # Extract event title
    title_tag = event.find(class_="em-card_title")
    title = title_tag.text.strip() if title_tag else None

    # Extract date and time information
    date_info_tag = event.find_all(class_="em-card_event-text")
    if date_info_tag:
        # Assuming the first <p> contains date and time info
        date_info = date_info_tag[0].text.strip() if date_info_tag else None
        location_tag = date_info_tag[1].find('a') if len(date_info_tag) > 1 else None
        location = location_tag.text.strip() if location_tag else None

        # Initialize event_info dictionary
        event_info = {
            'title': title,
            'date_info': date_info,
            'location': location,
            'image_url': None,
            'event_type': None,
            'date': None,
            'day_of_week': None,
            'start_time': None,
            'end_time': None
        }

        # Parse date info using regex
        if event_info['date_info']:
            date_match = re.match(
                r'(\w+), (\w+ \d+, \d{4})(?: (\d+(?::\d+)?[ap]m)(?: to (\d+(?::\d+)?[ap]m))?)?',
                event_info['date_info']
            )
            if date_match:
                original_date_str = date_match.group(2)
                # Convert the date to "YYYY-MM-DD" format
                try:
                    parsed_date = datetime.datetime.strptime(original_date_str, "%B %d, %Y")
                    formatted_date = parsed_date.strftime("%Y-%m-%d")
                    event_info['date'] = formatted_date
                except ValueError:
                    event_info['date'] = "Invalid date format"
                event_info['day_of_week'] = date_match.group(1)
                event_info['start_time'] = date_match.group(3)
                event_info['end_time'] = date_match.group(4)

    else:
        event_info['day_of_week'] = None
        event_info['date'] = None
        event_info['start_time'] = None
        event_info['end_time'] = None
        event_info['location'] = None

    # Extract image URL from the <div class="em-card_image">
    image_div = event.find(class_="em-card_image")

    if image_div:
        # Check for img tag directly
        image_tag = image_div.find("img")
        if image_tag:
            event_info['image_url'] = image_tag.get('src')  # Get src for the image URL

            # Optionally handle srcset for high-resolution images
            srcset = image_tag.get('srcset')
            if srcset:
                # Split srcset to get 2x resolution image URL if needed
                srcset_parts = srcset.split(',')
                if srcset_parts:
                    # Use the last entry (1x resolution)
                    event_info['image_url'] = srcset_parts[-1].split(' ')[0].strip()

    # Extract event type / first tag
    event_type_tag = event.find(class_="em-list_tags")
    event_info['event_type'] = event_type_tag.text.strip() if event_type_tag else None

    # Add the extracted event info to the list
    events.append(event_info)

    # Display event details
    print(f"Title: {event_info['title']}")
    print(f"Day: {event_info['day_of_week']}")
    print(f"Date: {event_info['date']}")
    print(f"Start Time: {event_info['start_time']}")
    print(f"End Time: {event_info['end_time']}")
    print(f"Location: {event_info['location']}")
    print(f"Event Type: {event_info['event_type']}")

    # Display the image (if available)
    if event_info['image_url']:
        print(f"Image URL: {event_info['image_url']}")
        display(Image(url=event_info['image_url']))

    print("-" * 40)


Title: Sunrise at the Mall
Day: Thu
Date: Invalid date format
Start Time: 6:45am
End Time: 8:30am
Location: Amsterdam Hall
Event Type: Student Life
----------------------------------------
Title: Cyberfuture Summit
Day: Thu
Date: Invalid date format
Start Time: 8am
End Time: 4pm
Location: Amentum
Event Type: Conference
----------------------------------------
Title: Envisioning India: Why is India Growing Fast But Not Creating Enough Jobs?
Day: Thu
Date: Invalid date format
Start Time: 9am
End Time: 10:30am
Location: Elliott School of International Affairs, Sigur Center Conference Room
Event Type: Academic
Image URL: https://localist-images.azureedge.net/photos/47658083121797/card/be36564e0c0ea74b36f1f2188880333052276a65.jpg


----------------------------------------
Title: Getting Started with Tableau
Day: Thu
Date: Invalid date format
Start Time: 9:30am
End Time: 11:30am
Location: Gelman Library
Event Type: None
Image URL: https://localist-images.azureedge.net/photos/45434560016329/card/da5fe9c85becaddb916d26eefd52a19758fa7a24.jpg


----------------------------------------
Title: 2025 OE: Legal Wellness Seminar
Day: Thu
Date: Invalid date format
Start Time: 10am
End Time: 11am
Location: None
Event Type: Health & Wellness
----------------------------------------
Title: A Tale of Two Houses
Day: Thu
Date: Invalid date format
Start Time: 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/47614343595670/card/318ae59ea438d46aa711fcb55172e093d256921a.jpg


----------------------------------------
Title: Irresistible: The Global Patterns of Ikat
Day: Thu
Date: Invalid date format
Start Time: 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/46703531502333/card/9f3494ee5d5e0690bf5d1702ca2c9f2331bc5b6d.jpg


----------------------------------------
Title: GW Dept. of Psychiatry - Grand Rounds
Day: Thu
Date: Invalid date format
Start Time: 10:30am
End Time: 12pm
Location: None
Event Type: 
Image URL: https://localist-images.azureedge.net/photos/47787952036812/card/a33a66809da43197ab52ce5058d7b3035b024005.jpg


----------------------------------------
Title: RCR Seminar Series: "Research Integrity Fireside Chat: Creating a Safe Research Environment"
Day: Thu
Date: Invalid date format
Start Time: 11am
End Time: 12pm
Location: University Student Center, 309
Event Type: Research
----------------------------------------
Title: The CCAS Career Doctor Is In
Day: Thu
Date: Invalid date format
Start Time: 11am
End Time: 1pm
Location: Philips Hall Front Desk, Philips Hall Lobby (next to elevators)
Event Type: Career & Professional Development
----------------------------------------
Title: Title IX Tabling
Day: Thu
Date: Invalid date format
Start Time: 11am
End Time: 2pm
Location: University Student Center, H Street Terrace
Event Type: Info Session
----------------------------------------
Title: GW Coders' Lunch and Learn: Open Source Showcase: GW Courses App
Day: Thu
Date: Invalid date format
Start Time: 11:30am
End Time: 12:30pm
Location: Science & Engineering Hall
Event Type: 
Image URL: https://lo

----------------------------------------
Title: A Cover Letter Is Like a Love Letter
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 1pm
Location: None
Event Type: Career & Professional Development
----------------------------------------
Title: Apple Day
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 2pm
Location: Kogan Plaza, located in front of Gelman Library at 2130 H Street, NW, Washington, D.C., 20052
Event Type: Arts & Culture
----------------------------------------
Title: Are They Getting It? Easy Ways to Gauge Understanding When Teaching
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 1pm
Location: None
Event Type: None
Image URL: https://localist-images.azureedge.net/photos/46372013701272/card/4640411b0fe531cecc380b9cbe4d393cf8026a98.jpg


----------------------------------------
Title: Chimeric Whims
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 5pm
Location: Smith Hall of Art (Academic Center), Gallery 102
Event Type: Arts & Culture
----------------------------------------
Title: Create Video Stories with Premiere Rush
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 1pm
Location: Gelman Library
Event Type: None
Image URL: https://localist-images.azureedge.net/photos/45434563570776/card/7de292b1700e6e35cf2d44d246c6820ac85934e4.jpg


----------------------------------------
Title: Federal Resume Writing: Presented by OPM
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 1pm
Location: None
Event Type: Career & Professional Development
----------------------------------------
Title: George Washington University Women's Tennis vs ITA Regional Championships
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 11:59pm
Location: Charlottesville, VA
Event Type: Athletics
----------------------------------------
Title: Pi Phi Pizookies
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: 4pm
Location: Strong Hall
Event Type: Student Life
----------------------------------------
Title: Semester in Washington Program Online Information Session
Day: Thu
Date: Invalid date format
Start Time: 12pm
End Time: None
Location: None
Event Type: Info Session
Image URL: https://localist-images.azureedge.net/photos/45871795516855/card/89cd57bd93520d8236efde50b547cee3288ef466.jpg


----------------------------------------


### Save Events


> NOTE: questions 4 and 5 are done for you 😸



4) Convert the entire list of `events` to a `pandas.DataFrame` object called `events_df`.

5) Write the `events_df` contents to **CSV file** called "events.csv" in the colab filesystem. Optionally download the CSV file and import into speadsheet software.

In [ ]:
from pandas import DataFrame

events_df = DataFrame(events)
events_df.head()

,title,date_info,location,image_url,event_type,date,day_of_week,start_time,end_time
0,Sunrise at the Mall,"Thu, Oct 17, 2024 6:45am to 8:30am",Amsterdam Hall,,Student Life,Invalid date format,Thu,6:45am,8:30am
1,Cyberfuture Summit,"Thu, Oct 17, 2024 8am to 4pm",Amentum,,Conference,Invalid date format,Thu,8am,4pm
2,Envisioning India: Why is India Growing Fast B...,"Thu, Oct 17, 2024 9am to 10:30am","Elliott School of International Affairs, Sigur...",https://localist-images.azureedge.net/photos/4...,Academic,Invalid date format,Thu,9am,10:30am
3,Getting Started with Tableau,"Thu, Oct 17, 2024 9:30am to 11:30am",Gelman Library,https://localist-images.azureedge.net/photos/4...,None,Invalid date format,Thu,9:30am,11:30am
4,2025 OE: Legal Wellness Seminar,"Thu, Oct 17, 2024 10am to 11am",None,,Health & Wellness,Invalid date format,Thu,10am,11am


In [ ]:
events_df.to_csv("events.csv")

## Part II

### Fetch Events Function



5-A) Write a custom function called `fetch_events`, which fetches the first page of events on a given day, and returns them as a list of dictionaries, where each dictionary contains the info mentioned in Question 2.
  + The function should accept an optional parameter called `selected_date` (using a default value of `None`). When a date string like "2023-10-20" is passed, the function should fetch and return events on the specified date. Otherwise if the `selected_date` parameter is omitted, such as by default, the function should return today's events.


> HINT: when we select a different date in the browser, the url will be something like: https://calendar.gwu.edu/calendar/day/2023/10/20


> HINT: use a default value of `None` for the `selected_date` parameter. Inside the function, check the parameter's value, and if one is present, adjust the request url to include something like "day/2023/10/20" at the end.

5-B) Revisit 5-A, and get all events for a given day (not just the first page of events).

> HINT: when we select a different page in the browser, the url will be something like: https://calendar.gwu.edu/calendar/day/2023/10/20/3 for page 3. Use a loop to loop through the pages and increment the page counter until there are no more pages.

6) Implement tests for the `fetch_events` function, using the examples below. Also write your own tests to describe what it means for the function to be working properly. Make sure your tests pass.


```python
from datetime import date

# example tests:
events = fetch_events()
assert any(events)
assert events[0]["date"] == str(date.today())

events = fetch_events("2023-10-20")
assert any(events)
assert events[0]["date"] == "2023-10-20"
```

Function definition:

In [ ]:


# # FUNCTION DEFINITION:

# def fetch_events(selected_date=None):
#     events = []

#     # TODO: fetch events

#     # TODO: parse events

#     return events



In [ ]:
from datetime import date

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Image
from datetime import datetime, date
import re

# FUNCTION DEFINITION:
def fetch_events(selected_date=None):
    events = []


    if selected_date:
        date_obj = datetime.strptime(selected_date, "%Y-%m-%d")
        url_date = f"{date_obj.year}/{date_obj.month}/{date_obj.day}"
    else:
        today = date.today()
        url_date = f"{today.year}/{today.month}/{today.day}"


    url = f"https://calendar.gwu.edu/calendar/day/{url_date}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')


    for event in soup.find_all(class_="em-card"):
        # Extract event title
        title_tag = event.find(class_="em-card_title")
        title = title_tag.text.strip() if title_tag else None


        date_info_tag = event.find_all(class_="em-card_event-text")
        if date_info_tag:
            date_info = date_info_tag[0].text.strip() if date_info_tag else None
            location_tag = date_info_tag[1].find('a') if len(date_info_tag) > 1 else None
            location = location_tag.text.strip() if location_tag else None

            # Decompose the date info into specific fields
            date_info_parts = date_info.split(",")
            day_of_week = date_info_parts[0].strip() if len(date_info_parts) > 0 else None
            date_str = date_info_parts[1].strip() if len(date_info_parts) > 1 else None

            # Convert the date to YYYY-MM-DD format
            if date_str:
                month_day_str = date_str.strip()
                year = date.today().year
                date_obj = datetime.strptime(f"{month_day_str} {year}", "%b %d %Y")
                date_formatted = date_obj.strftime("%Y-%m-%d")
            else:
                date_formatted = None

            # Handling times, if present
            if "to" in date_info:

                time_info = date_info.split("to")
                start_time = time_info[0].split(",")[-1].strip() if time_info[0] else None
                end_time = time_info[1].strip() if len(time_info) > 1 else None
            else:
                start_time = None
                end_time = None
        else:
            day_of_week = None
            date_formatted = None
            start_time = None
            end_time = None
            location = None

        # Extract image URL from the <div class="em-card_image">
        image_div = event.find(class_="em-card_image")
        image_url = None

        if image_div:
            # Check for img tag directly
            image_tag = image_div.find("img")
            if image_tag:
                image_url = image_tag.get('src')  # Get src for the image URL

                # Optionally handle srcset for high-resolution images
                srcset = image_tag.get('srcset')
                if srcset:
                    # Split srcset to get 2x resolution image URL if needed
                    srcset_parts = srcset.split(',')
                    if srcset_parts:
                        # Use the last entry (1x resolution)
                        image_url = srcset_parts[-1].split(' ')[0].strip()

        # Extract event type / first tag
        event_type_tag = event.find(class_="em-list_tags")
        event_type = event_type_tag.text.strip() if event_type_tag else None

        # Add the extracted event info to the list as a dictionary
        events.append({
            'title': title,
            'day_of_week': day_of_week,
            'date': date_formatted,
            'start_time': start_time,
            'end_time': end_time,
            'location': location,
            'image_url': image_url,
            'event_type': event_type
        })

        # Display event details
        print(f"Title: {title}")
        print(f"Day: {day_of_week}")
        print(f"Date: {date_formatted}")
        print(f"Start Time: {start_time}")
        print(f"End Time: {end_time}")
        print(f"Location: {location}")
        print(f"Event Type: {event_type}")

        # Display the image (if available)
        if image_url:
            print(f"Image URL: {image_url}")
            display(Image(url=image_url))

        print("-" * 40)

    return events


Example tests:

In [ ]:

events = fetch_events()
print("Today's Date:", str(date.today()))
print("First Event Date:", events[0]["date"])
assert events[0]["date"] == str(date.today())

Title: Alumni & Families Weekend 2024
Day: Fri
Date: 2024-10-18
Start Time: None
End Time: None
Location: None
Event Type: Alumni
----------------------------------------
Title: Celebrate Fall!
Day: Fri
Date: 2024-10-18
Start Time: 2024 9am
End Time: 11am
Location: Merriweather Lobby
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47690782256649/card/e71f075e20a53b6a15a714ff48a7f890ea49628e.jpg


----------------------------------------
Title: 2025 OE: Critical Illness & Hospital Indemnity Overview
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: None
Event Type: Health & Wellness
----------------------------------------
Title: A Tale of Two Houses
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/47614343595670/card/318ae59ea438d46aa711fcb55172e093d256921a.jpg


----------------------------------------
Title: Beyond Boba: Taiwan’s Culinary Culture In A Global Context
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11:30am
Location: Elliott School of International Affairs, room 505
Event Type: 
----------------------------------------
Title: Chemistry Seminar: Mirza Galib, Howard University
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: Science & Engineering Hall, B1220
Event Type: Lectures & Speakers
Image URL: https://localist-images.azureedge.net/photos/47729117112843/card/c651a1417499c7c3f9cc729c0575d7272993b450.jpg


----------------------------------------
Title: Irresistible: The Global Patterns of Ikat
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/46703531502333/card/9f3494ee5d5e0690bf5d1702ca2c9f2331bc5b6d.jpg


----------------------------------------
Title: Meet Current GW eOTD Students
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: None
Event Type: Info Session
Image URL: https://localist-images.azureedge.net/photos/47557977862954/card/845410628c5914dcb70b0ebcf94659cb7c486819.jpg


...

----------------------------------------
Title: Artistic Wellness: GW Art Therapy Clinic
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: Military Community Center
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47540305515785/card/906b82fd97f1657557819a25026b0c3e5cb85692.jpg


----------------------------------------
Title: Chimeric Whims
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 5pm
Location: Smith Hall of Art (Academic Center), Gallery 102
Event Type: Arts & Culture
----------------------------------------
Title: Fuel & Flourish
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1:30pm
Location: District House Lounge
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47611077378059/card/e5122aeffd57793a14ec61642503671e6e7db42e.jpg


----------------------------------------
Title: GW Dining Student Advisory Panel Meeting
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: District House, B206
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/47499972175738/card/feefbed6a157bb98111178bb106d61f5c9b349bc.jpg


----------------------------------------
Title: GW Dining Student Advisory Panel Meeting
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: District House B206
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47504881305848/card/bf07121d7c75172c7aef8eaa1f8d6e9807d424d4.jpg


----------------------------------------
Today's Date: 2024-10-18
First Event Date: 2024-10-18


Your own tests:

In [ ]:
# Fetch all events
events = fetch_events()

# Filter events happening on Friday
events_on_friday = [event for event in events if event["day_of_week"] == "Fri"]

# Debugging and testing the results
print("Fetched Events on Friday:", events_on_friday)
assert any(events_on_friday), "No events found on Friday."

# Check if the first event's day is Friday
print("First Event Day:", events_on_friday[0]["day_of_week"])
assert events_on_friday[0]["day_of_week"] == "Fri", "First event doesn't happen on Friday."


Title: Alumni & Families Weekend 2024
Day: Fri
Date: 2024-10-18
Start Time: None
End Time: None
Location: None
Event Type: Alumni
----------------------------------------
Title: Celebrate Fall!
Day: Fri
Date: 2024-10-18
Start Time: 2024 9am
End Time: 11am
Location: Merriweather Lobby
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47690782256649/card/e71f075e20a53b6a15a714ff48a7f890ea49628e.jpg


----------------------------------------
Title: 2025 OE: Critical Illness & Hospital Indemnity Overview
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: None
Event Type: Health & Wellness
----------------------------------------
Title: A Tale of Two Houses
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/47614343595670/card/318ae59ea438d46aa711fcb55172e093d256921a.jpg


----------------------------------------
Title: Beyond Boba: Taiwan’s Culinary Culture In A Global Context
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11:30am
Location: Elliott School of International Affairs, room 505
Event Type: 
----------------------------------------
Title: Chemistry Seminar: Mirza Galib, Howard University
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: Science & Engineering Hall, B1220
Event Type: Lectures & Speakers
Image URL: https://localist-images.azureedge.net/photos/47729117112843/card/c651a1417499c7c3f9cc729c0575d7272993b450.jpg


----------------------------------------
Title: Irresistible: The Global Patterns of Ikat
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 5pm
Location: The George Washington Museum and the Textile Museum
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/46703531502333/card/9f3494ee5d5e0690bf5d1702ca2c9f2331bc5b6d.jpg


----------------------------------------
Title: Meet Current GW eOTD Students
Day: Fri
Date: 2024-10-18
Start Time: 2024 10am
End Time: 11am
Location: None
Event Type: Info Session
Image URL: https://localist-images.azureedge.net/photos/47557977862954/card/845410628c5914dcb70b0ebcf94659cb7c486819.jpg


...

----------------------------------------
Title: Artistic Wellness: GW Art Therapy Clinic
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: Military Community Center
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47540305515785/card/906b82fd97f1657557819a25026b0c3e5cb85692.jpg


----------------------------------------
Title: Chimeric Whims
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 5pm
Location: Smith Hall of Art (Academic Center), Gallery 102
Event Type: Arts & Culture
----------------------------------------
Title: Fuel & Flourish
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1:30pm
Location: District House Lounge
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47611077378059/card/e5122aeffd57793a14ec61642503671e6e7db42e.jpg


----------------------------------------
Title: GW Dining Student Advisory Panel Meeting
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: District House, B206
Event Type: Arts & Culture
Image URL: https://localist-images.azureedge.net/photos/47499972175738/card/feefbed6a157bb98111178bb106d61f5c9b349bc.jpg


----------------------------------------
Title: GW Dining Student Advisory Panel Meeting
Day: Fri
Date: 2024-10-18
Start Time: 2024 12pm
End Time: 1pm
Location: District House B206
Event Type: Student Life
Image URL: https://localist-images.azureedge.net/photos/47504881305848/card/bf07121d7c75172c7aef8eaa1f8d6e9807d424d4.jpg


----------------------------------------
Fetched Events on Friday: [{'title': 'Alumni & Families Weekend 2024', 'day_of_week': 'Fri', 'date': '2024-10-18', 'start_time': None, 'end_time': None, 'location': None, 'image_url': '', 'event_type': 'Alumni'}, {'title': 'Celebrate Fall!', 'day_of_week': 'Fri', 'date': '2024-10-18', 'start_time': '2024 9am', 'end_time': '11am', 'location': 'Merriweather Lobby', 'image_url': 'https://localist-images.azureedge.net/photos/47690782256649/card/e71f075e20a53b6a15a714ff48a7f890ea49628e.jpg', 'event_type': 'Student Life'}, {'title': '2025 OE: Critical Illness & Hospital Indemnity Overview', 'day_of_week': 'Fri', 'date': '2024-10-18', 'start_time': '2024 10am', 'end_time': '11am', 'location': None, 'image_url': '', 'event_type': 'Health & Wellness'}, {'title': 'A Tale of Two Houses', 'day_of_week': 'Fri', 'date': '2024-10-18', 'start_time': '2024 10am', 'end_time': '5pm', 'location': 'The George Washington Museum and the Textile Museum', 'image_url': '

## Exploration / Scratch Work

Optionally use this section for scratch work. Its contents will not be evaluated. Make sure the "Solution" section contains any code you want to be evaluated.

### Date Parsing Function

In [ ]:

def date_parser(date_info:str):

    # TODO: parse the date string

    return {
        "weekday": "TODO",
        "date": "TODO",
        "start": "TODO",
        "end": "TODO",
    }


In [ ]:
from datetime import datetime

def date_parser(date_str):
    # Check if the date string includes a time range
    if "to" in date_str:
        # Case with start and end times
        parts = date_str.split(" to ")
        start_time = datetime.strptime(parts[0], "%a, %b %d, %Y %I%p")

        # Parse the end time from the second part
        end_time = datetime.strptime(parts[1], "%I:%M%p").strftime("%I:%M%p").lower()

        return {
            "weekday": start_time.strftime("%a"),  # Short weekday name ("Mon")
            "date": start_time.strftime("%Y-%m-%d"),
            "start": start_time.strftime("%I:%M%p").lower(),  # Start time in 12-hour format
            "end": end_time  # End time in 12-hour format
        }
    else:
        # Case with only the date, no times
        start_time = datetime.strptime(date_str, "%a, %b %d, %Y")  # Parse date only

        return {
            "weekday": start_time.strftime("%a"),  # Short weekday name ("Mon")
            "date": start_time.strftime("%Y-%m-%d"),
            "start": None,  # No start time
            "end": None  # No end time
        }


d1 = "Mon, Oct 2, 2023 10am to 11:30am"
parsed_result_d1 = date_parser(d1)
print(parsed_result_d1)

assert parsed_result_d1 == {
    "weekday": "Mon",  # Short weekday name
    "date": "2023-10-02",
    "start": "10:00am",  # Start time in 12-hour format
    "end": "11:30am"  # End time in 12-hour format
}


d2 = "Mon, Oct 2, 2023"
parsed_result_d2 = date_parser(d2)
print(parsed_result_d2)

assert parsed_result_d2 == {
    "weekday": "Mon",  # Short weekday name
    "date": "2023-10-02",
    "start": None,  # No start time
    "end": None  # No end time
}


{'weekday': 'Mon', 'date': '2023-10-02', 'start': '10:00am', 'end': '11:30am'}
{'weekday': 'Mon', 'date': '2023-10-02', 'start': None, 'end': None}
